In [ ]:
import os

In [ ]:
def load_MAG_and_divide(dir_genomes, MAG_TO_VIEW):

    contigs = {}
    contigs_rev = {}

    pre = "ACTG"
    post = "TGAC"

    os.chdir(dir_genomes)
    with open(MAG_TO_VIEW) as f: #### SELECT SPECIFIC MAG
        v = f.readlines()
        f.seek(0)
        start = -1
        curr_h = ""
        curr_f = ""
        for line in v:
            if line.startswith(">"):
                contigs[curr_h] = curr_f

                curr_f = ""
                curr_h = line.strip()
            else:
                curr_f += line.strip()
        contigs[curr_h] = curr_f

        start = -1
        curr_h = ""
        curr_f = ""
        curr_f_rev = ""
        for line in v:
            if line.startswith(">"):
                compl = curr_f.maketrans(pre,post)
                fasta_compl = curr_f.translate(compl)
                curr_f_rev = fasta_compl[::-1]
                contigs_rev[curr_h] = curr_f_rev
                curr_f = ""
                curr_h = line.strip()
            else:
                curr_f += line.strip()

        compl = curr_f.maketrans(pre,post)
        fasta_compl = curr_f.translate(compl)
        curr_f_rev = fasta_compl[::-1]
        contigs_rev[curr_h] = curr_f_rev
    
    contigs.pop("")
    contigs_rev.pop("")
    
    return contigs, contigs_rev

def rewrite_MAG_removing_genes(MAG_TO_REWRITE, contigs, dir_output, DELETION_SIZE, dict_of_genes_to_remove):
    '''
    
    '''
    
    os.chdir(dir_output)

    if not MAG_TO_REWRITE in os.listdir():
        g = open(MAG_TO_REWRITE, "a")
        g.close()

    f = open(MAG_TO_REWRITE, "w")
    for fasta, seq in contigs.items():
        seq_to_write_by_lines = ""
        f.write(fasta+"\n")

        if fasta in dict_of_genes_to_remove.keys():
            
            print(fasta) # debug

            START = int(dict_of_genes_to_remove[fasta][0])
            STRAND = dict_of_genes_to_remove[fasta][1]
            GENE_TO_REMOVE = dict_of_genes_to_remove[fasta][2]
            GENE_LENGHT = len(GENE_TO_REMOVE)

            if DELETION_SIZE == "small":

                PORTION_TO_REMOVE = round(GENE_LENGHT/100*30)
                PORTION_TO_REMOVE_MINUS = GENE_LENGHT - PORTION_TO_REMOVE

                if STRAND == "+":

                    gene_seq_removed = GENE_TO_REMOVE[:PORTION_TO_REMOVE]
                    gene_remove_index = seq.index(gene_seq_removed)

                    gene_seq_maintained = GENE_TO_REMOVE[PORTION_TO_REMOVE:]
                    #print(gene_seq_removed) #debug
                    #print(gene_seq_maintained) #debug

                    seq_to_write = ""
                    seq_to_write += seq[:gene_remove_index]
                    seq_to_write += seq[gene_remove_index+PORTION_TO_REMOVE:]

                if STRAND == "-":
                    gene_seq_removed = GENE_TO_REMOVE[PORTION_TO_REMOVE_MINUS:]
                    gene_remove_index = seq.index(gene_seq_removed)
                    gene_seq_maintained = GENE_TO_REMOVE[:PORTION_TO_REMOVE_MINUS]
                    #print(gene_seq_removed) #debug
                    #print(gene_seq_maintained) #debug
                    #print(seq[(START+GENE_LENGHT):]) # debug

                    seq_to_write = ""
                    seq_to_write += seq[:gene_remove_index]
                    seq_to_write += seq[START+GENE_LENGHT:]

            elif DELETION_SIZE == "big":

                PORTION_TO_REMOVE = round(GENE_LENGHT/100*70)
                PORTION_TO_REMOVE_MINUS = GENE_LENGHT - PORTION_TO_REMOVE

                if STRAND == "+":
                    gene_seq_removed = GENE_TO_REMOVE[:PORTION_TO_REMOVE]
                    gene_remove_index = seq.index(gene_seq_removed)
                    gene_seq_maintained = GENE_TO_REMOVE[PORTION_TO_REMOVE:]
                    #print(gene_seq_removed) #debug
                    #print(gene_seq_maintained) #debug

                    seq_to_write = ""
                    seq_to_write += seq[:gene_remove_index]
                    seq_to_write += seq[gene_remove_index+PORTION_TO_REMOVE:]

                if STRAND == "-":

                    gene_seq_removed = GENE_TO_REMOVE[PORTION_TO_REMOVE_MINUS:]
                    gene_remove_index = seq.index(gene_seq_removed)

                    gene_seq_maintained = GENE_TO_REMOVE[:PORTION_TO_REMOVE_MINUS]
                    #print(gene_seq_removed) #debug
                    #print(gene_seq_maintained) #debug

                    seq_to_write = ""
                    seq_to_write += seq[:gene_remove_index]
                    seq_to_write += seq[START+GENE_LENGHT:]

            for nt in seq_to_write:
                if len(seq_to_write_by_lines) == 60:
                    f.write(seq_to_write_by_lines+"\n")
                    seq_to_write_by_lines = ""
                    pass
                seq_to_write_by_lines += nt
            f.write(seq_to_write_by_lines+"\n")

        else:
            for nt in seq:
                if len(seq_to_write_by_lines) == 60:
                    f.write(seq_to_write_by_lines+"\n")
                    seq_to_write_by_lines = ""
                    pass
                seq_to_write_by_lines += nt
            f.write(seq_to_write_by_lines+"\n")
    f.close()

def rewrite_Genome_removing_genes(GENOME_TO_REWRITE, contigs, dir_output, DELETION_SIZE, dict_of_genes_to_remove):
    '''
    
    '''
    
    os.chdir(dir_output)
    if not GENOME_TO_REWRITE in os.listdir():
        g = open(GENOME_TO_REWRITE, "a")
        g.close()

    actual_position = -1

    f = open(GENOME_TO_REWRITE, "w")
    fasta = ">"+GENOME_TO_REWRITE.replace(".fna", "").replace(".fasta", "")
    seq = contigs[fasta]
    seq_to_write_by_lines = ""
    f.write(fasta+"\n")
    gene_positions = []

    for gene_pos in dict_of_genes_to_remove.keys():
        START = gene_pos
        gene_positions.append(START)
    #print(gene_positions) #debug

    seq_to_write = ""
    pos = -1
    n_nt = -1

    for nt in seq[:-1]:
        #print(nt)
        n_nt += 1
        if not n_nt > pos:
            continue
        pos += 1
        
        #try:
        if not str(pos) in gene_positions:
            #print(seq[actual_position])
            seq_to_write += seq[pos]

        elif str(pos) in gene_positions:
            GENE_TO_REMOVE = dict_of_genes_to_remove[str(pos)][1]
            GENE_LENGHT = len(GENE_TO_REMOVE)
            STRAND = dict_of_genes_to_remove[str(pos)][0]
            #print("gene_len: "+str(GENE_LENGHT)) # debug

        # different effect for strand
            if STRAND == "+":

            # different effect for the DELETION_SIZE
                if DELETION_SIZE == "small":
                    PORTION_TO_REMOVE = round(GENE_LENGHT/100*30)
                    PORTION_TO_REMOVE_MINUS = GENE_LENGHT - PORTION_TO_REMOVE
                elif DELETION_SIZE == "big":
                    PORTION_TO_REMOVE = round(GENE_LENGHT/100*70)
                    PORTION_TO_REMOVE_MINUS = GENE_LENGHT - PORTION_TO_REMOVE

                gene_seq_removed = GENE_TO_REMOVE[:PORTION_TO_REMOVE]
                gene_remove_index = seq.index(gene_seq_removed)
                gene_seq_maintained = GENE_TO_REMOVE[PORTION_TO_REMOVE:]
                #print(gene_seq_removed)
                #print("maint: "+gene_seq_maintained)

        # different effect for strand
            elif STRAND == "-":

            # different effect for the DELETION_SIZE
                if DELETION_SIZE == "small":
                    PORTION_TO_REMOVE = round(GENE_LENGHT/100*30)
                    PORTION_TO_REMOVE_MINUS = GENE_LENGHT - PORTION_TO_REMOVE
                elif DELETION_SIZE == "big":
                    PORTION_TO_REMOVE = round(GENE_LENGHT/100*70)
                    PORTION_TO_REMOVE_MINUS = GENE_LENGHT - PORTION_TO_REMOVE

                gene_seq_removed = GENE_TO_REMOVE[PORTION_TO_REMOVE_MINUS:]
                gene_remove_index = seq.index(gene_seq_removed)
                gene_seq_maintained = GENE_TO_REMOVE[:PORTION_TO_REMOVE_MINUS]
                #print(gene_seq_removed)
                #print("maint: "+gene_seq_maintained)

            seq_to_write += gene_seq_maintained
            pos = pos+GENE_LENGHT-1
            print(pos) # debug
        #except:
            #pass

    for char in seq_to_write:
        if len(seq_to_write_by_lines) == 80:
            f.write(seq_to_write_by_lines+"\n")
            seq_to_write_by_lines = ""
            pass
        seq_to_write_by_lines += char
    f.write(seq_to_write_by_lines+"\n")
    f.close()

def translate_sequence(nucleotidic_seq):
    '''
    Translate genomic-retrieved sequence in AA alphabeth
    --------------------------------------------------
    INPUT:  nucleotidic sequence
    OUTPUT: aminoacidic sequence
    '''

    aaseq = ""

    t11 = {"TTT":"F", "TTC":"F", "TTA":"L", "TTG":"L",
           "TCT":"S", "TCC":"S", "TCA":"S", "TCG":"S",
           "TAT":"Y", "TAC":"Y", "TAA":"*", "TAG":"*",
           "TGT":"C", "TGC":"C", "TGA":"*", "TGG":"W",
           "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
           "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P",
           "CAT":"H", "CAC":"H", "CAA":"Q", "CAG":"Q",
           "CGT":"R", "CGC":"R", "CGA":"R", "CGG":"R",
           "ATT":"I", "ATC":"I", "ATA":"I", "ATG":"M",
           "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
           "AAT":"N", "AAC":"N", "AAA":"K", "AAG":"K",
           "AGT":"S", "AGC":"S", "AGA":"R", "AGG":"R",
           "GTT":"V", "GTC":"V", "GTA":"V", "GTG":"V",
           "GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A",
           "GAT":"D", "GAC":"D", "GAA":"E", "GAG":"E",
           "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G",}

    codons = [nucleotidic_seq[i:i+3] for i in range(0, len(nucleotidic_seq), 3)]
    for el in codons:
        if len(el) < 3:
            codons.remove(el)

    for codon in codons:
        tr_codon = t11[codon]
        aaseq += tr_codon

    return aaseq

def load_prodigal_genes(dir_prodigal, PRODIGAL_GENE_FILE):
    genes = {}
    
    os.chdir(dir_prodigal)
    with open(PRODIGAL_GENE_FILE) as f:
        v = f.readlines()
        f.seek(0)
        start = -1
        curr_h = ""
        curr_f = ""
        for line in v:
            if line.startswith(">"):
                genes[curr_h] = curr_f

                curr_f = ""
                curr_h = line.strip().split(" ")[0]
            else:
                curr_f += line.strip()
        genes[curr_h] = curr_f

    genes.pop("")
    return genes

# gene boundaries search (for gene removal purposes)

In [ ]:
git_directory = os.getcwd()+"/"

In [ ]:
dir_genomes = git_directory+"5_complete_genomes/"
dir_prodigal = dir_genomes+"prodigal/"


PRODIGAL_GENE_FILE = "NZ_CP053946.1.fna"
MAG_TO_VIEW = "NZ_CP053946.1.fna"

contigs, contigs_rev = load_MAG_and_divide(dir_genomes, MAG_TO_VIEW)

GENE_OF_INTEREST = ">"+"NZ_CP053946.1_3603"

In [ ]:
genes = load_prodigal_genes(dir_prodigal, PRODIGAL_GENE_FILE)
SEQ_gene_of_interest = genes[GENE_OF_INTEREST]
LENGHT_gene_of_interest = len(SEQ_gene_of_interest)

for fasta, seq in contigs.items():
    if SEQ_gene_of_interest in seq:
        bound_1 = seq.index(SEQ_gene_of_interest)
        bound_2 = seq.index(SEQ_gene_of_interest)+LENGHT_gene_of_interest

        #print(fasta+" - genomic boundaries: "+str(bound_1)+"-"+str(bound_2))
        
        #print(seq[bound_1:bound_2])

for fasta, seq in contigs_rev.items():
    if SEQ_gene_of_interest in seq:
        lenght_contig = len(seq)

        bound_1 = seq.index(SEQ_gene_of_interest)
        bound_2 = seq.index(SEQ_gene_of_interest)+LENGHT_gene_of_interest

        bound_2_plus = lenght_contig-bound_1
        bound_1_plus = lenght_contig-bound_2
        
        bound_1 = bound_1_plus
        bound_2 = bound_2_plus
                
        #print(fasta+" (MINUS_STRAND)"+" - genomic boundaries: "+str(bound_1)+"-"+str(bound_2))
        
        pre = "ACTG"
        post = "TGAC"
        compl = contigs[fasta].maketrans(pre,post)
        fasta_compl = contigs[fasta].translate(compl)
        rev_seq = fasta_compl[::-1]
        
        #print(contigs[fasta][bound_1:bound_2])

## gene removal for MAG sequences

In [ ]:
LIST_GENES_TO_REMOVE = [
    ">",
    ">",
    ">"
] # UPDATED EACH TIME WITH GENE NAMES FROM README.md

dir_genomes = git_directory+"0_control_group_results/"
dir_prodigal = dir_genomes+"prodigal/"

PRODIGAL_GENE_FILE = "NC_000913.3.fna"
MAG_TO_VIEW = "NC_000913.3.fna"

In [ ]:
contigs, contigs_rev = load_MAG_and_divide(dir_genomes, MAG_TO_VIEW)
genes = load_prodigal_genes(dir_prodigal, PRODIGAL_GENE_FILE)

dict_of_genes_to_remove = {}

for GENE in LIST_GENES_TO_REMOVE:
    SEQ_gene_of_interest = genes[GENE]

    LENGHT_gene_of_interest = len(SEQ_gene_of_interest)

    n += 1

    for fasta, seq in contigs.items():
        if SEQ_gene_of_interest in seq:
            bound_1 = seq.index(SEQ_gene_of_interest)
            bound_2 = seq.index(SEQ_gene_of_interest)+LENGHT_gene_of_interest

            dict_of_genes_to_remove.update({fasta:[str(bound_1),"+",seq[bound_1:bound_2]]})

    for fasta, seq in contigs_rev.items():
        if SEQ_gene_of_interest in seq:
            lenght_contig = len(seq)

            bound_1 = seq.index(SEQ_gene_of_interest)
            bound_2 = seq.index(SEQ_gene_of_interest)+LENGHT_gene_of_interest

            bound_2_plus = lenght_contig-bound_1
            bound_1_plus = lenght_contig-bound_2

            bound_1 = bound_1_plus
            bound_2 = bound_2_plus

            pre = "ACTG"
            post = "TGAC"
            compl = contigs[fasta].maketrans(pre,post)
            fasta_compl = contigs[fasta].translate(compl)
            rev_seq = fasta_compl[::-1]

            dict_of_genes_to_remove.update({fasta:[str(bound_1),"-",contigs[fasta][bound_1:bound_2]]})

In [ ]:
MAG_TO_REWRITE = MAG_TO_VIEW
contigs = contigs

#####################
# TEST_1   bigINDELS#
#####################
dir_output = dir_genomes.replace("5_complete_genomes", "6_start_complete_bigINDELS")
DELETION_SIZE = "big"

rewrite_MAG_removing_genes(MAG_TO_REWRITE, contigs, dir_output, DELETION_SIZE, dict_of_genes_to_remove)

#####################
# TEST_2 smallINDELS#
#####################
dir_output = dir_genomes.replace("5_complete_genomes", "7_start_complete_smallINDELS")
DELETION_SIZE = "small"

rewrite_MAG_removing_genes(MAG_TO_REWRITE, contigs, dir_output, DELETION_SIZE, dict_of_genes_to_remove)

## removal for Genomes

In [ ]:
LIST_GENES_TO_REMOVE = [
    ">",
    ">",
    ">"
] # UPDATED EACH TIME WITH GENE NAMES FROM README.md

dir_genomes = git_directory+"5_complete_genomes/"
dir_prodigal = dir_genomes+"prodigal/"

PRODIGAL_GENE_FILE = "NZ_CP053946.1.fna"
MAG_TO_VIEW = "NZ_CP053946.1.fna"

In [ ]:
contigs, contigs_rev = load_MAG_and_divide(dir_genomes, MAG_TO_VIEW)
genes = load_prodigal_genes(dir_prodigal, PRODIGAL_GENE_FILE)

dict_of_genes_to_remove = {}

n = -1
for GENE in LIST_GENES_TO_REMOVE:
    SEQ_gene_of_interest = genes[GENE]

    LENGHT_gene_of_interest = len(SEQ_gene_of_interest)

    n += 1

    for fasta, seq in contigs.items():
        if SEQ_gene_of_interest in seq:

            bound_1 = seq.index(SEQ_gene_of_interest)
            bound_2 = seq.index(SEQ_gene_of_interest)+LENGHT_gene_of_interest

            dict_of_genes_to_remove.update({str(bound_1):["+",seq[bound_1:bound_2]]})

    for fasta, seq in contigs_rev.items():
        if SEQ_gene_of_interest in seq:
            lenght_contig = len(seq)

            bound_1 = seq.index(SEQ_gene_of_interest)
            bound_2 = seq.index(SEQ_gene_of_interest)+LENGHT_gene_of_interest

            bound_2_plus = lenght_contig-bound_1
            bound_1_plus = lenght_contig-bound_2

            bound_1 = bound_1_plus
            bound_2 = bound_2_plus

            pre = "ACTG"
            post = "TGAC"
            compl = contigs[fasta].maketrans(pre,post)
            fasta_compl = contigs[fasta].translate(compl)
            rev_seq = fasta_compl[::-1]

            dict_of_genes_to_remove.update({str(bound_1):["-",contigs[fasta][bound_1:bound_2]]})

In [ ]:
GENOME_TO_REWRITE = MAG_TO_VIEW
contigs = contigs

#####################
# TEST_1   bigINDELS#
#####################
dir_output = dir_genomes.replace("5_complete_genomes", "6_start_complete_bigINDELS")
DELETION_SIZE = "big"

rewrite_Genome_removing_genes(GENOME_TO_REWRITE, contigs, dir_output, DELETION_SIZE, dict_of_genes_to_remove)

#####################
# TEST_2 smallINDELS#
#####################
dir_output = dir_genomes.replace("5_complete_genomes", "7_start_complete_smallINDELS")
DELETION_SIZE = "small"

rewrite_Genome_removing_genes(GENOME_TO_REWRITE, contigs, dir_output, DELETION_SIZE, dict_of_genes_to_remove)

# check MAG final results sequences - gene presence & BLASTp purposes

In [ ]:
os.chdir(git_directory+"3_MAG_confrontation_and_final_results")

sequences = {}

with open("MAG_FINAL_RESULTS.tsv") as f:
    for line in f.readlines()[2:]:
        line = line.strip().split("\t")

        MAG = line[0]
        KO = line[1]
        seq_0 = line[2]
        xseq_0 = line[3]
        seq_1 = line[4]
        xseq_1 = line[5]
        seq_2 = line[6]
        xseq_2 = line[7]
        
        if not seq_0 == "NaN":
            fasta_0 = ">"+MAG+"_"+KO+"__0"
            sequences.update({fasta_0:seq_0})
        if not seq_1 == "NaN":
            fasta_1 = ">"+MAG+"_"+KO+"__1"
            sequences.update({fasta_1:seq_1})
        if not seq_2 == "NaN":
            fasta_2 = ">"+MAG+"_"+KO+"__2"
            sequences.update({fasta_2:seq_2})

#print(sequences)

# Prodigal gene presence/lack thereof

In [ ]:
# code to generate "tests/3_MAG_confrontation_and_final_results/other_data/hits_found_in_0_prodigal.txt"
dir_prodigal = git_directory+"0_control_group_results/prodigal/"

for el in os.listdir(dir_prodigal):
    if el.endswith(".fna"):
        PRODIGAL_GENE_FILE = el
        print("++"+PRODIGAL_GENE_FILE+"\n")
        try:
            genes = load_prodigal_genes(dir_prodigal, PRODIGAL_GENE_FILE)

            for gene in genes:

                for header, hmmhit_seq in sequences.items():
                    if hmmhit_seq in genes[gene]:
                        print(header)
                        print("\t"+gene) # to check if the same 
            print("")
        except:
            pass

In [ ]:
# code to generate "tests/3_MAG_confrontation_and_final_results/other_data/hits_found_in_2_prodigal.txt"
dir_prodigal = git_directory+"2_smallINDELS_group_results/prodigal/"

for el in os.listdir(dir_prodigal):
    if el.endswith(".fna"):
        PRODIGAL_GENE_FILE = el
        print("++"+PRODIGAL_GENE_FILE+"\n")
        try:
            genes = load_prodigal_genes(dir_prodigal, PRODIGAL_GENE_FILE)

            for gene in genes:

                for header, hmmhit_seq in sequences.items():
                    if hmmhit_seq in genes[gene]:
                        print(header)
                        print("\t"+gene) # to check if the same 
            print("")
        except:
            pass

In [ ]:
# code to generate "tests/3_MAG_confrontation_and_final_results/other_data/hits_found_in_1_prodigal.txt"
dir_prodigal = git_directory+"1_bigINDELS_group_results/prodigal/"

for el in os.listdir(dir_prodigal):
    if el.endswith(".fna"):
        PRODIGAL_GENE_FILE = el
        print("++"+PRODIGAL_GENE_FILE+"\n")
        try:
            genes = load_prodigal_genes(dir_prodigal, PRODIGAL_GENE_FILE)

            for gene in genes:

                for header, hmmhit_seq in sequences.items():
                    if hmmhit_seq in genes[gene]:
                        print(header)
                        print("\t"+gene) # to check if the same 
            print("")
        except:
            pass

# check Complete genomes final results sequences - gene presence & BLASTp purposes

In [ ]:
os.chdir(git_directory+"7_COMPLETE_GENOMES_confrontation_and_final_results")

sequences = {}

with open("COMPLETE_FINAL_RESULTS.tsv") as f:
    for line in f.readlines()[2:]:
        line = line.strip().split("\t")

        GENOME = line[0]
        KO = line[1]
        seq_0 = line[2]
        xseq_0 = line[3]
        seq_1 = line[4]
        xseq_1 = line[5]
        seq_2 = line[6]
        xseq_2 = line[7]
        
        if not seq_0 == "NaN":
            fasta_0 = ">"+GENOME+"_"+KO+"__0"
            sequences.update({fasta_0:seq_0})
        if not seq_1 == "NaN":
            fasta_1 = ">"+GENOME+"_"+KO+"__1"
            sequences.update({fasta_1:seq_1})
        if not seq_2 == "NaN":
            fasta_2 = ">"+GENOME+"_"+KO+"__2"
            sequences.update({fasta_2:seq_2})

#print(sequences)

# Prodigal gene presence/lack thereof

In [ ]:
# code to generate "tests/7_COMPLETE_GENOMES_confrontation_and_final_results/other_data/hits_found_in_4_prodigal.txt"
dir_prodigal = git_directory+"4_complete_control_group_results/prodigal/"

for el in os.listdir(dir_prodigal):
    if el.endswith(".fna"):
        PRODIGAL_GENE_FILE = el
        print("++"+PRODIGAL_GENE_FILE+"\n")
        try:
            genes = load_prodigal_genes(dir_prodigal, PRODIGAL_GENE_FILE)

            for gene in genes:

                for header, hmmhit_seq in sequences.items():
                    if hmmhit_seq in genes[gene]:
                        print(header)
                        print("\t"+gene) # to check if the same 
            print("")
        except:
            pass

In [ ]:
# code to generate "tests/7_COMPLETE_GENOMES_confrontation_and_final_results/other_data/hits_found_in_6_prodigal.txt"
dir_prodigal = git_directory+"6_complete_smallINDELS_group_results/prodigal/"

for el in os.listdir(dir_prodigal):
    if el.endswith(".fna"):
        PRODIGAL_GENE_FILE = el
        print("++"+PRODIGAL_GENE_FILE+"\n")
        try:
            genes = load_prodigal_genes(dir_prodigal, PRODIGAL_GENE_FILE)

            for gene in genes:

                for header, hmmhit_seq in sequences.items():
                    if hmmhit_seq in genes[gene]:
                        print(header)
                        print("\t"+gene) # to check if the same 
            print("")
        except:
            pass

In [ ]:
# code to generate "tests/7_COMPLETE_GENOMES_confrontation_and_final_results/other_data/hits_found_in_5_prodigal.txt"
dir_prodigal = git_directory+"5_complete_bigINDELS_group_results/prodigal/"

for el in os.listdir(dir_prodigal):
    if el.endswith(".fna"):
        PRODIGAL_GENE_FILE = el
        print("++"+PRODIGAL_GENE_FILE+"\n")
        try:
            genes = load_prodigal_genes(dir_prodigal, PRODIGAL_GENE_FILE)

            for gene in genes:

                for header, hmmhit_seq in sequences.items():
                    if hmmhit_seq in genes[gene]:
                        print(header)
                        print("\t"+gene) # to check if the same 
            print("")
        except:
            pass